## Ingest the CSV file into Big Query:

In [1]:
dataset_id = "discogs_staging"

In [2]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'earnest-keep-266820:discogs_staging'
already exists.


In [8]:
!bq --location=US load --autodetect --max_bad_records 10000 --source_format=NEWLINE_DELIMITED_JSON \
{dataset_id}.Artist \
'gs://cs327e_discogs/final_artist.json'

Waiting on bqjob_r2ada6d6ab5c5259a_000001718ad4f7d5_1 ... (65s) Current status: DONE   
Warnings encountered during job execution:

[gs://cs327e_discogs/final_artist.json] Error while reading data, error message: JSON parsing error in row starting at position 1306886702: Could not convert value to string. Field: Val; Value: 12

[gs://cs327e_discogs/final_artist.json] Error while reading data, error message: JSON parsing error in row starting at position 1307860638: Could not convert value to string. Field: Val; Value: 3.141592653589793

[gs://cs327e_discogs/final_artist.json] Error while reading data, error message: JSON parsing error in row starting at position 1308142322: Could not convert value to string. Field: Val; Value: 3.141592653589793

[gs://cs327e_discogs/final_artist.json] Error while reading data, error message: JSON parsing error in row starting at position 1308249490: Could not convert value to string. Field: Val; Value: 3.141592653589793

[gs://cs327e_discogs/final_arti

In [1]:
%%bigquery
select count(*) from discogs_staging.Artist

,f0_
0,6789773


In [1]:
%%bigquery
select * from discogs_staging.Artist limit 3

,groups,data_quality,urls,realname,name,aliases,profile,id,members,namevariations,images
0,None,{'Val': 'Complete and Correct'},{'url': [{'Val': 'http://www.facebook.com/enie...,{'Val': 'Sandra Urbančíková'},{'Val': 'Eniesa'},"{'name': [{'Val': 'Sandra Urbančíková', 'Zid':...",{'Val': 'Slovak vocalist.'},{'Val': 5121860},None,None,"{'image': [{'Zwidth': 600, 'Zheight': 715, 'Zu..."
1,None,{'Val': 'Needs Minor Changes'},{'url': [{'Val': 'http://www.anexartisi.gr/abo...,None,{'Val': 'Abolish Authority'},None,{'Val': 'Η μπάντα σχηματίστηκε τον ον Ιούλιο τ...,{'Val': 2673594},"{'id': [{'Val': 424836}, {'Val': 689921}, {'Va...",None,"{'image': [{'Zwidth': 356, 'Zheight': 280, 'Zu..."
2,None,{'Val': 'Complete and Correct'},None,None,{'Val': 'Pet Show Noise'},None,{'Val': 'PE / Depressive / Black Noise project...,{'Val': 1704485},"{'id': [{'Val': 2133891}], 'name': [{'Val': 'R...",None,"{'image': [{'Zwidth': 311, 'Zheight': 186, 'Zu..."


In [3]:
!bq --location=US load --autodetect --max_bad_records 10000 --source_format=NEWLINE_DELIMITED_JSON \
{dataset_id}.Label \
'gs://cs327e_discogs/labels_final.json'

Waiting on bqjob_r232d50ab00ae62a4_000001718b63846d_1 ... (34s) Current status: DONE   
Warnings encountered during job execution:

[gs://cs327e_discogs/labels_final.json] Error while reading data, error message: JSON parsing error in row starting at position 37743: Could not convert value to string. Field: Val; Value: 6277

[gs://cs327e_discogs/labels_final.json] Error while reading data, error message: JSON parsing error in row starting at position 90979: Could not convert value to string. Field: Val; Value: 33

[gs://cs327e_discogs/labels_final.json] Error while reading data, error message: JSON parsing error in row starting at position 258367: Could not convert value to string. Field: Val; Value: 130701

[gs://cs327e_discogs/labels_final.json] Error while reading data, error message: JSON parsing error in row starting at position 449920: Could not convert value to string. Field: Val; Value: 1210

[gs://cs327e_discogs/labels_final.json] Error while reading data, error message: JSON 

In [2]:
%%bigquery
select count(*) from discogs_staging.Label

,f0_
0,1506930


In [2]:
%%bigquery
select * from discogs_staging.Label limit 3

,data_quality,id,contactinfo,parentLabel,urls,name,sublabels,profile,images
0,{'Val': 'Complete and Correct'},{'Val': 640},{'Val': 'musicZretroactiverecords.us'},None,{'url': [{'Val': 'http://www.retroactiverecord...,{'Val': 'Retroactive'},None,{'Val': 'A DETROIT label founded in 1990 by [a...,"{'image': [{'Zwidth': 150, 'Zheight': 184, 'Zu..."
1,{'Val': 'Complete and Correct'},{'Val': 1096},{'Val': 'Ant-Zen p.o. box 1257 93135 Lappers...,None,"{'url': [{'Val': 'http://www.ant-zen.com/'}, {...",{'Val': 'Ant-Zen'},"{'label': [{'Val': 'Bazooka Joe', 'Zid': 13928...",{'Val': 'Full name: Ant-Zen audio & visual art...,"{'image': [{'Zwidth': 170, 'Zheight': 127, 'Zu..."
2,{'Val': 'Complete and Correct'},{'Val': 3295},{'Val': 'Taaach! Edition Chrome Postfach 10 ...,None,None,{'Val': 'Taaach! Recordings'},"{'label': [{'Val': 'Edition Taaach!', 'Zid': 7...",{'Val': 'German Trance Label Distributed by I...,"{'image': [{'Zwidth': 158, 'Zheight': 55, 'Zur..."


### Before creating the modeled tables, we will remove rows which have been tagged as 'entirely incorrect' in the database.

In [3]:
%%bigquery
create table discogs_staging.Artist_Clean as
select * from discogs_staging.Artist where data_quality.val <> 'Entirely Incorrect' or data_quality.val is null

""


In [4]:
%%bigquery
create table discogs_staging.Label_Clean as
select * from discogs_staging.Label where data_quality.val <> 'Entirely Incorrect' or data_quality.val is null

""
